In [1]:
import pandas as pd

import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.stem import LancasterStemmer 

nltk.download("stopwords")

import string
import re

import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB

from sklearn.linear_model import SGDClassifier

import numpy as np

from scipy import sparse


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/natashamathur/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#train = pd.read_csv("s3://no-hate/data/train.csv")

In [3]:
test = pd.read_csv("s3://advancedml-koch-mathur-hinkson/test.csv")

In [4]:
train = pd.read_csv("s3://advancedml-koch-mathur-hinkson/train.csv")

KeyboardInterrupt: 

In [5]:
train = pd.read_csv('train.csv')

In [55]:
ls = LancasterStemmer()
ps = PorterStemmer() 

sw = set(stopwords.words('english'))
sw.add('')

def clean_text(text, stemming=None, remove_sw = True):

    t = text.replace("-", " ").split(" ")
    
    t = [w.lower() for w in t]
    
    if remove_sw == True:
        t = [w for w in t if w not in sw]
    
    if stemming == None:
        pass;
    elif stemming == "Porter":
        t = [ps.stem(w) for w in t]
    elif stemming == "Lancaster":
        t = [ls.stem(w) for w in t]
    else:
        print("Please enter a valid stemming type")
        
    t = [w.strip(string.punctuation) for w in t]


    return ' '.join(t)

In [56]:
def add_text_cleaning_cols(df):
    
    df['split'] = df["comment_text"].apply(lambda x: x.split(" "))
    df['cleaned_w_stopwords'] = df["comment_text"].apply(clean_text,args=(None,False),)

    df['cleaned_no_stem'] = df["comment_text"].apply(clean_text,)
    df['cleaned_porter'] = df["comment_text"].apply(clean_text,args=("Porter",),)
    df['cleaned_lancaster'] = df["comment_text"].apply(clean_text,args=("Lancaster",),)

    df['perc_upper'] = df["comment_text"].apply(lambda x: round((len(re.findall(r'[A-Z]',x)) / len(x)), 3))

    df['num_exclam'] = df["comment_text"].apply(lambda x:(len(re.findall(r'!',x))))
    
    df['num_words'] = df["split"].apply(lambda x: len(x))
    
    df['perc_stopwords'] = round((df.num_words - df['cleaned_no_stem'].apply(lambda x: len(x)))/df.num_words,3) 
    
    df['num_upper_words'] = df["split"].apply(lambda x: sum(map(str.isupper, x)) 
)
    
    

In [57]:
train_sample = train.head(10000)
add_text_cleaning_cols(train_sample)

In [58]:
train_sample.columns

Index(['id', 'target', 'comment_text', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count', 'split',
       'cleaned_w_stopwords', 'cleaned_no_stem', 'cleaned_porter',
       'cleaned_lancaster', 'perc_upper', 'num_exclam', 'num_words',
       'perc_stopwords', 'num_upper_words'],
      dtype='object')

In [59]:
addtl_cols = ['perc_upper', 'num_exclam', 'num_words', 'perc_stopwords', 'num_upper_words']
#addtl_cols = ['perc_upper']

In [60]:
train_sample.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,split,cleaned_w_stopwords,cleaned_no_stem,cleaned_porter,cleaned_lancaster,perc_upper,num_exclam,num_words,perc_stopwords,num_upper_words
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,"[This, is, so, cool., It's, like,, 'would, you...",this is so cool it's like would you want your ...,cool like would want mother read this really g...,cool like would want mother read this realli g...,cool like would want moth read this real gre i...,0.030,1,19,-2.421,0
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,"[Thank, you!!, This, would, make, my, life, a,...",thank you this would make my life a lot less a...,thank you would make life lot less anxiety ind...,thank you would make life lot less anxieti ind...,thank you would mak lif lot less anxy inducing...,0.026,3,22,-2.545,0
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,"[This, is, such, an, urgent, design, problem;,...",this is such an urgent design problem kudos to...,urgent design problem kudos taking on impressive,urgent design problem kudo take on impressive,urg design problem kudo tak on impressive,0.023,1,16,-2.000,0
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,"[Is, this, something, I'll, be, able, to, inst...",is this something i'll be able to install on m...,something i'll able install site releasing it,someth i'll abl instal site releas it,someth i'll abl instal site releas it,0.036,0,17,-1.647,0
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,"[haha, you, guys, are, a, bunch, of, losers.]",haha you guys are a bunch of losers,haha guys bunch losers,haha guy bunch losers,hah guy bunch losers,0.000,0,8,-1.750,0


In [3]:
def run_model(model_df, train_perc=.80, addtl_feats =[''], model_type = "Gauss", 
              num_iter = 10, should_print=False, see_inside=False, comments="comment_text"):
    
    train_start = 0
    train_end = round(model_df.shape[0]*train_perc) 

    test_start = train_end
    test_end = model_df.shape[0]
    
    X_all = model_df[comments].values
    y_all = model_df['target'].values
    
    # tokenizing text
    count_vect = CountVectorizer()
    X_all_counts = count_vect.fit_transform(X_all.astype('U'))
    #print(X_all_counts.shape)

    # calculating frequencies
    tf_transformer = TfidfTransformer(use_idf=False).fit(X_all_counts)
    X_all_tfidf = tf_transformer.transform(X_all_counts)

    print(X_all_tfidf.shape)
    
    if addtl_feats != ['']: # combine non-text and text features if necessary
        print("here")
        others_all = model_df[addtl_feats].values.reshape(-1,1)
        newfeatures_all = sparse.hstack((X_all_tfidf, others_all.astype(float))).tocsr()
    else:
        newfeatures_all = X_all_tfidf
    
    
    X_train = newfeatures_all[train_start:train_end]
    y_train = model_df[train_start:train_end]['target'].values
    y_train=y_train.astype('int')

    X_test = newfeatures_all[test_start:test_end]
    y_test = model_df[test_start:test_end]['target'].values
    
    
    if model_type == 'Multi':
        clf = MultinomialNB().fit(X_train, y_train)
    if model_type == "Gauss":
        clf = GaussianNB().fit(X_train, y_train) 
   
    predicted = clf.predict(X_test)
    accuracy = np.mean(predicted == y_test)

    if should_print == True:

        print("The accuracy on the test set is {}%.".format(round(accuracy*100,2)))    
    
    if see_inside == True:
        return clf, accuracy, X_all_counts, X_all_tfidf
    else:
        return clf, accuracy


In [4]:
run_model(train_sample, should_print=True)

NameError: name 'train_sample' is not defined

In [5]:
for c in addtl_cols:
    print(c)
    print(run_model(train_sample, comments = comm, addtl_feats = [c], should_print=True))
    print()

NameError: name 'addtl_cols' is not defined

In [61]:
for comm in ['cleaned_w_stopwords', 'cleaned_no_stem', 'cleaned_porter',
       'cleaned_lancaster']:
    print(comm)
    print(run_model(train_sample, comments = comm, should_print=True))
    print()

cleaned_w_stopwords
(10000, 25628)
The accuracy on the test set is 71.0%.
(MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True), 0.71)

cleaned_no_stem
(10000, 25626)
The accuracy on the test set is 71.0%.
(MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True), 0.71)

cleaned_porter
(10000, 23409)
The accuracy on the test set is 71.0%.
(MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True), 0.71)

cleaned_lancaster
(10000, 22247)
The accuracy on the test set is 71.0%.
(MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True), 0.71)



In [ ]:
    `